# BigQuery ML

[Original Post](https://towardsdatascience.com/how-to-use-k-means-clustering-in-bigquery-ml-to-understand-and-describe-your-data-better-c972c6f5733b)

BigQuery ML now supports unsupervised learning — you can apply the K-Means algorithm to group your data into clusters. Unlike supervised machine learning, which is about predictive analytics, unsupervised learning is about descriptive analytics — it’s about understanding your data so that you can make data-driven decisions.
![Nice Picture](https://cdn-images-1.medium.com/max/1600/1*-t8gKksp_rjn5gM7V0MuIw.jpeg)
K-Means clustering is an unsupervised learning algorithm that can help you understand your data and provide descriptive labels to your it. Photo by Randy Fath on Unsplash

## What are you clustering?

First, determine what you are clustering and why you are doing it. Suppose you are a retailer who has data on customer transactions. There are several ways you could do the clustering on this dataset, and it depends on what you want to do with the clusters.

- You could find natural groups amongst your customers. This is called customer segmentation. Data you use to do the customer segmentation might include things like which store they visited, what items they bought, how much they paid, etc. You would try to understand what these groups of customers are like (these are called personas) so that you can design items that appeal to members of one of those groups.
- You could find natural groups amongst the items purchased. These are called product groups. You would cluster items based on who purchased them, when they were purchased, which store they were purchased at, etc. You might try to learn characteristics of a product group so that you can learn how to reduce _cannibalization_ or improve _cross-selling_.
In both these cases, you are using clustering as a heuristic to help you make decisions — it’s too hard to design individualized products or understand product interactions, so you design for groups of customers or groups of items.

Note that for the specific use case of product recommendations (recommending products to customers or targeting customers for a product), it is better train an ML model to do that exact thing, using a [collaborative approach such as WALS](https://towardsdatascience.com/how-to-build-a-collaborative-filtering-model-for-personalized-recommendations-using-tensorflow-and-b9a77dc1320). But for other decisions for which there is no readily available predictive analytics approach, clustering might give you a way to make a data-driven decision.

Setting up clustering problem
In order to use clustering well, you need to do four things:

1. Identify what field you are clustering on. CustomerId? ItemId? Then, cluster the data on attributes of that field.
1. Find which cluster a given customer/item/etc. belongs to.
1. Understand something about a cluster’s attributes.
1. Use this understanding to make decisions.

In this article, I’ll demonstrate the clustering approach on London bike share data. Let’s say that we make a lot of decisions around the design and stocking of bike share stations and we would like to understand the different types of stations we have in our system so that we can make these decisions in a data-driven way.

We need to do these four things:

1. Cluster bicycle rental stations. The field we need to cluster is station_name, and we need to find attributes of a station in order to do so.
1. Find which cluster each station belongs to.
1. Understand the characteristics of each cluster.
1. Use this understanding to make data-driven decisions.
But before all that, we first have to gather the data we need.

## 0. Gather data
The data we will use consists of bike rentals in London and it’s a public dataset in the EU region, so make sure to create a dataset in the EU region called demos_eu if you want to follow along with my queries.

We want to cluster stations based on these factors:

- Duration of rentals
- Number of trips per day
- Number of bike racks
- Distance from city center
We often make decisions such as stocking the station (i.e. making sure the station has bikes for rent) on weekdays differently from weekends. So, we are actually going to cluster instances whose “primary key” is the combination of station_name and isweekday.

Here’s a query that’ll pull together the relevant data:

In [ ]:
service_account_file='../../service-account.json'

In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=service_account_file
%load_ext google.cloud.bigquery
import json
with open(service_account_file, 'r') as f:
    sa = json.load(f)
project_id=sa['project_id']
os.environ['GOOGLE_CLOUD_PROJECT']=project_id

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
%%bigquery

#%sql -d standard

WITH hs AS (
SELECT 
h.start_station_name as station_name,
IF(EXTRACT(DAYOFWEEK FROM h.start_date) = 1 OR
   EXTRACT(DAYOFWEEK FROM h.start_date) = 7, "weekend", "weekday") as isweekday,
h.duration,
s.bikes_count,
ST_DISTANCE(ST_GEOGPOINT(s.longitude, s.latitude), 
            ST_GEOGPOINT(-0.1, 51.5))/1000 as distance_from_city_center
FROM `bigquery-public-data.london_bicycles.cycle_hire` as h
JOIN `bigquery-public-data.london_bicycles.cycle_stations` as s
ON h.start_station_id = s.id
WHERE h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP) AND CAST('2016-01-01 00:00:00' AS TIMESTAMP)
),
stationstats AS (
SELECT 
station_name,
isweekday,
AVG(duration) as duration,
COUNT(duration) as num_trips,
MAX(bikes_count) as bikes_count,
MAX(distance_from_city_center) as distance_from_city_center
FROM hs
GROUP BY station_name, isweekday
)
SELECT * 
from stationstats
ORDER BY distance_from_city_center ASC

We are extracting data on cycle hires (start_station_name, weekend/weekday, duration) in 2015 (see the WHERE clause) and joining it against station information (bike count, distance-from-city-center).

Then, we compute attributes of the station in stationstats (the average duration of rides, number of trips) and pass through the station attributes (bike count, distance-from-city-center). Our dataset looks like this:
![For Original Post](https://cdn-images-1.medium.com/max/1600/1*x98lMTTsP7mL4FKYNaL63w.png)

Dataset that we will use for clustering; The primary key for our data is the combination of station_name and isweekday.

## 1. BigQuery ML Clustering
Doing the clustering simply involves adding a CREATE MODEL statement to the SELECT query above and removing the “id” fields in our data:

In [ ]:
%%bigquery

CREATE OR REPLACE MODEL bqml_tutorial_eu.london_station_clusters
OPTIONS(model_type='kmeans', num_clusters=4, standardize_features = true) AS
WITH hs AS (
SELECT 
h.start_station_name as station_name,
IF(EXTRACT(DAYOFWEEK FROM h.start_date) = 1 OR
   EXTRACT(DAYOFWEEK FROM h.start_date) = 7, "weekend", "weekday") as isweekday,
h.duration,
s.bikes_count,
ST_DISTANCE(ST_GEOGPOINT(s.longitude, s.latitude), 
            ST_GEOGPOINT(-0.1, 51.5))/1000 as distance_from_city_center
FROM `bigquery-public-data.london_bicycles.cycle_hire` as h
JOIN `bigquery-public-data.london_bicycles.cycle_stations` as s
ON h.start_station_id = s.id
WHERE h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP) AND CAST('2016-01-01 00:00:00' AS TIMESTAMP)
),
stationstats AS (
SELECT 
station_name,
isweekday,
AVG(duration) as duration,
COUNT(duration) as num_trips,
MAX(bikes_count) as bikes_count,
MAX(distance_from_city_center) as distance_from_city_center
FROM hs
GROUP BY station_name, isweekday
)
SELECT * except(station_name, isweekday)
from stationstats

This query processed 1.2 GB and took 54 seconds. The model schema lists the 4 factors that were used in the clustering:
![result set](https://cdn-images-1.medium.com/max/1600/1*W-AmKNznarOnjQvD7d1Y8A.png)

The K-Means model schema
Note that we have specified the desired number of clusters (num_clusters=4) in my CREATE MODEL and removed the factors (station_name and isweekday) that we don’t want to cluster on. Standardizing features is a good option to include if your features have different dynamic ranges (bikes_count is in the 10–50 range while num_trips is in the thousands, so I am doing that).

If we leave out the number of clusters, BigQuery ML will choose a reasonable default based on the total number of rows in the training dataset. We could also do hyperparameter tuning to find a good number. Essentially, we will run the clustering for different values of num_clusters, find the error measure, and pick the point at which the graph plateaus out. The error measure can be obtained using:



In [ ]:
%%bigquery

SELECT davies_bouldin_index 
FROM ML.EVALUATE(MODEL bqml_tutorial_eu.london_station_clusters)

The lower the number, the better the clustering.

## 2. Which cluster does a station belong to?
Which cluster does a particular station belong to? You get that using ML.PREDICT. Here’s a query to find the cluster of every station that has ‘Kennington’ in its name:

In [ ]:
%%bigquery

WITH hs AS (
SELECT 
h.start_station_name as station_name,
IF(EXTRACT(DAYOFWEEK FROM h.start_date) = 1 OR
   EXTRACT(DAYOFWEEK FROM h.start_date) = 7, "weekend", "weekday") as isweekday,
h.duration,
s.bikes_count,
ST_DISTANCE(ST_GEOGPOINT(s.longitude, s.latitude), 
            ST_GEOGPOINT(-0.1, 51.5))/1000 as distance_from_city_center
FROM `bigquery-public-data.london_bicycles.cycle_hire` as h
JOIN `bigquery-public-data.london_bicycles.cycle_stations` as s
ON h.start_station_id = s.id
WHERE h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP) AND CAST('2016-01-01 00:00:00' AS TIMESTAMP)
),
stationstats AS (
SELECT 
station_name,
isweekday,
AVG(duration) as duration,
COUNT(duration) as num_trips,
MAX(bikes_count) as bikes_count,
MAX(distance_from_city_center) as distance_from_city_center
FROM hs
GROUP BY station_name, isweekday
)

SELECT * except(nearest_centroids_distance) 
FROM ML.PREDICT(MODEL bqml_tutorial_eu.london_station_clusters, 
(SELECT * FROM stationstats WHERE REGEXP_CONTAINS(station_name, 'Kennington')))

This yields:
![results](https://cdn-images-1.medium.com/max/1600/1*FttwiK30j9V5-IrbcYpFLw.png)

Which cluster do the Kennington stations belong to?
## 3. Examine clusters
You can look at the cluster centroids — essentially the values of the 4 factors in the model schema using:

In [ ]:
%%bigquery

SELECT * FROM ML.CENTROIDS(MODEL bqml_tutorial_eu.london_station_clusters)
ORDER BY centroid_id

With a little bit of SQL manipulation, we can pivot the above table:

In [ ]:
%%bigquery

WITH T AS (
SELECT 
centroid_id,
ARRAY_AGG(STRUCT(feature AS name, ROUND(numerical_value,1) AS value) ORDER BY centroid_id) AS cluster
FROM ML.CENTROIDS(MODEL bqml_tutorial_eu.london_station_clusters)
GROUP BY centroid_id
)
SELECT
CONCAT('Cluster#', CAST(centroid_id AS STRING)) AS centroid,
(SELECT value from unnest(cluster) WHERE name = 'duration') AS duration,
(SELECT value from unnest(cluster) WHERE name = 'num_trips') AS num_trips,
(SELECT value from unnest(cluster) WHERE name = 'bikes_count') AS bikes_count,
(SELECT value from unnest(cluster) WHERE name = 'distance_from_city_center') AS distance_from_city_center
FROM T
ORDER BY centroid_id ASC

This yields:
![results](https://cdn-images-1.medium.com/max/1600/1*GaQAmD391aSo9Dl2MD42Pw.png)

Clusters’ attributes
To visualize this table, click on “Explore in Data Studio” and select “Table with bars”. Make the centroid column the “dimension” and the remaining columns the metrics. Then you will get:


Cluster attributes, visualized in Data Studio
This allows us to interpret the clusters. Essentially, Cluster #1 consists of suburban stations from which people go for long rides, Cluster #2 consists of other suburban stations, Cluster #3 consists of busy city stations, and Cluster #4 consists of not-so-busy city stations.

Based on these characteristics and some knowledge of London, we can come up with descriptive names for these clusters. Cluster 1 would probably be “Day trips”, Cluster 2 would be “Bedroom communities”, Cluster 3 would be “Tourist areas”, and Cluster 4 would be “Business district”.

## 4. Make data-driven decisions
Let’s use these clusters to make different decisions.

- We just got funding and can expand the bike racks. Which stations should we put extra capacity in? If we didn’t have the clustering data, we’d be tempted to go with stations with lots of trips and not enough bikes — stations in Cluster #3. But we have done the clustering, and discovered that this group of stations serves mostly tourists. They don’t vote, so we’ll put the extra capacity in Cluster #2 (bedroom communities).
- We need to experiment with a new type of lock. Which cluster of stations should we do this experiment on? The business district stations seems logical, and sure enough those are the stations with lots of bikes and which are busy enough to support an A/B test.
- We want to stock some stations with racing bikes. Which ones? Cluster 1, the stations that serve people who going on day trips out of the city.
Obviously, we could have made these decisions individually by doing custom data analysis each time. But clustering the stations, coming up with descriptive names, and using the names to make decisions is much simpler and more explainable.

The fact that you can do all this using just SQL? That’s pretty cool.